In [28]:
import sys
import platform

if platform.machine() == 'arm64':
    pth = "/Users/saksh.menon/Documents/GitHub/C-RNN-approach"
elif platform.machine() == 'x86_64':
    pth = '/home/sakshmeno/Documents/GitHub/C-RNN-approach'

sys.path.append(pth)
from Utils.OneHotGen import *


import Utils
import numpy as np


In [20]:
# df = dataframe_gen.dataframe_init()
df_obj = Utils.dataframe_gen
df = df_obj.dataframe_init()

In [37]:
OHE_df = OHV_init()

NameError: name 'dataframe_gen' is not defined